In [ ]:
import adif_io
import pandas as pd
import requests
import json
from time import sleep
import os
import subprocess
from pathlib import Path

In [ ]:
pd.set_option('display.max_columns', 500)

# Read adif

In [ ]:
qsos_raw, adif_header = adif_io.read_from_file("POTA-Test.adi")

# The QSOs are probably sorted by QSO time already, but make sure:
for qso in qsos_raw:
    qso["t"] = adif_io.time_on(qso)
qsos_raw_sorted = sorted(qsos_raw, key = lambda qso: qso["t"])

In [ ]:
qsos = pd.DataFrame(qsos_raw_sorted)
# qsos.info()

In [ ]:
qsos

# Query HamDB

In [ ]:
responses = []
for i, call in enumerate(qsos['CALL'].to_list()):
    response = requests.get("http://api.hamdb.org/v1/{call}/json/CDR".format(call=call))
    responses.append(pd.DataFrame(json.loads(response.text)['hamdb']['callsign'], index=[i]))
    sleep(0.1)

In [ ]:
addresses = pd.concat(responses)

In [ ]:
qsl_info = qsos.merge(addresses, left_on='CALL', right_on='call')

In [ ]:
while qsl_info.shape[0] % 4 > 0:
    qsl_info = qsl_info.append(pd.Series(), ignore_index=True)

In [ ]:
qsl_info.loc[qsl_info['FREQ'].astype(float)>100, ['Rig', 'Ant']] = ['UV5R', 'DB Slim Jim']
qsl_info.loc[qsl_info['FREQ'].astype(float)<100, ['Rig', 'Ant']] = ['TX-500', 'EFHW 40m']

In [ ]:
qsl_info['Front'] = 0
qsl_info.loc[range(0, len(qsl_info), 4), 'Front'] = 1

In [ ]:
for c in ['addr2', 'NAME', 'fname', 'name']:
    qsl_info.loc[:, c] = qsl_info.loc[:, c].str.title()

In [ ]:
qsl_info['date'] = pd.to_datetime(qsl_info['t']).dt.strftime("%d-%b-%Y")
qsl_info['time'] = pd.to_datetime(qsl_info['t']).dt.strftime("%H%M")
qsl_info['zip'] = qsl_info['zip'].str[:5]
qsl_info.fillna(' ', inplace=True)

In [ ]:
qsl_info

In [ ]:
out = qsl_info[['Front', 'CALL', 'date', 'time', 'BAND', 'FREQ', 'MODE', 'MY_GRIDSQUARE', 'fname', 'name'
              , 'RST_RCVD', 'RST_SENT', 'TX_PWR', 'addr1', 'addr2', 'state', 'zip'
              , 'Rig', 'Ant', 'MY_SIG_INFO']]

In [ ]:
out

# Write CSV

In [ ]:
out.to_csv('latex/addresses.csv', index_label='index')

# Build PDF

In [ ]:
cwd = os.getcwd()

In [ ]:
os.chdir(Path(cwd, 'latex'))

In [ ]:
subprocess.Popen("/Library/TeX/texbin/pdflatex {}".format('test.tex'), shell=True)

In [ ]:
os.chdir(cwd)